In [28]:
import pandas as pd
from collections import Counter

data = pd.read_csv("estadao_news.csv")

In [29]:
data['articles'] = data['titulo']  + ' ' + data['conteudo']

In [30]:
to_lower_case = lambda text: text.lower()

In [31]:
tokenize = lambda row: row.split()

In [32]:
data['articles'] = data['articles'].apply(to_lower_case)
data['tokens'] = data['articles'].apply(tokenize)

In [33]:
counter = lambda row: Counter(row)

In [81]:
def count_frequence(article, token):
    counter = Counter(article)
    return counter[token]

def summarize(matrix_of_tokens, docIds):
    index = {}
    for i in range(len(matrix_of_tokens)):
        for token in matrix_of_tokens[i]:
            if token in index.keys():
                #index[token].append((docIds[i], count_frequence(matrix_of_tokens[i], token)))
                index[token].append(docIds[i])
            else:
                #index[token] = [(docIds[i], count_frequence(matrix_of_tokens[i], token))]
                index[token] = [docIds[i]]
    
    return index

In [82]:
inverted_index = summarize(data['tokens'], data['idNoticia'])

In [83]:
# Tests
def search(query):
    elements = query.split()
    operation = elements[1]
    
    result = []
    if operation == 'OR':
        result = list(inverted_index.get(elements[0].lower(), []))
        result.extend(list(inverted_index.get(elements[2].lower(), [])))
    else:
        result = set(inverted_index.get(elements[0].lower(), [])).intersection(inverted_index.get(elements[2].lower(), []))
    
    return set(result)

assert len(search("debate OR presidencial")) == 1770
assert len(search("debate AND presidencial")) == 201

assert len(search("presidenciáveis OR corruptos")) == 164
assert len(search("presidenciáveis AND corruptos")) == 0

assert len(search("Belo OR Horizonte")) == 331
assert len(search("Belo AND Horizonte")) == 242

In [96]:
def conjunctive_search(query):
    elements = query.split()
    elements = filter(lambda x: x != 'AND', elements)
    
    index = {}
    for element in elements:
        index[len(inverted_index[element])] = element
    
    ordered_frequence = sorted(index.keys())
    
    result = set(inverted_index[index[ordered_frequence[0]]])
    for i in range(1, len(ordered_frequence)):
        result = result.intersection(inverted_index[index[ordered_frequence[i]]])
        
    return result
        

print(len(conjunctive_search("inflação foi culpa do pt")))
print(len(conjunctive_search("inflação foi culpa do temer")))

2
